In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import pandas as pd
import numpy as np

from src import Household

In [6]:
buyer = Household(name="가구 1", kwh=420).set_rate("단일", "기타")

print("현재요금", buyer.basic, buyer.elec_rate, buyer.basic + buyer.elec_rate)

현재요금 7300 61852.0 69152.0


In [7]:
seller = Household(name="가구 2", kwh=150).set_rate("단일", "기타")

print("현재요금", seller.basic, seller.elec_rate, seller.basic + seller.elec_rate)

현재요금 910 13995.0 14905.0


In [42]:
# 서비스 소개 알고리즘 정리

# 판매 권장 고객 ? 누진 1단계 사용 고객
print("고객 {}은 판매 권장고객입니다.\n".format(seller.name))

# 여기서 예측기가 붙을 거임
predict_kwh = 170
print("현재 사용량은 {}kWh이며, 이번달 예측 사용량은 {}kWh입니다.\n".format(
    seller.kwh,
    predict_kwh
))

# 거래 추천
# 아마 가장 단순하게는 누진 2단계 시작 구간인 200kWh - 예측 사용량 170kWh를 빼는 형식으로 추천이 될거임
reco_trading_kwh = 200 - predict_kwh
print("사용량 {}kwh의 거래를 추천해드립니다.\n".format(reco_trading_kwh))

# 현재 상태에서 거래를 진행했을 경우의 가격을 우선적으로 계산하겠지?
after_seller = Household(name="가구 2", kwh=seller.kwh + reco_trading_kwh).set_rate("단일", "기타")
print("거래 진행 시, 고객님의 사용량은 {}kWh가 되며,\n".format(after_seller.kwh)
     + "전기요금계는 {}원에서 {}원으로 {}원 증가합니다.\n\n".format(
         seller.basic + seller.elec_rate,
         after_seller.basic + after_seller.elec_rate,
         (after_seller.basic + after_seller.elec_rate) - (seller.basic + seller.elec_rate)
     ))

# 그러면 아마 이 고객이 가장 이득을 얻을 수 있는 구매자를 찾아낼 것임
print("고객 {}와의 거래를 추천드립니다.".format(buyer.name))
# 그리고 buyer의 가격도 우선 측정하겠지?
after_buyer = Household(name="가구 1", kwh=buyer.kwh - reco_trading_kwh).set_rate("단일", "기타")
print("거래 진행 시, 구매자의 사용량은 {}kWh가 되며,\n".format(after_buyer.kwh)
     + "전기요금계는 {}원에서 {}원으로 {}원 감소합니다.\n\n".format(
         buyer.basic + buyer.elec_rate,
         after_buyer.basic + after_buyer.elec_rate,
         (buyer.basic + buyer.elec_rate) - (after_buyer.basic + after_buyer.elec_rate)
     ))

# 이 때 상품의 가치는 고객 2가 최소로는 받아야하는 금액 (eq. 올라간 금액만큼의 금액) 부터
# 고객 1과의 거래 시, 최대로 요구할 수 있는 가격이 온다. (eq. 구매자에게 내려간 금액, 구매자가 지불해도 구매자가 손해를 보지 않는 금액)
min_val = (after_seller.basic + after_seller.elec_rate) - (seller.basic + seller.elec_rate)
max_val = (buyer.basic + buyer.elec_rate) - (after_buyer.basic + after_buyer.elec_rate)
print("거래 진행 시, 거래사용량 {}kWh의 상품 가치는 {}원 ~ {}원 입니다.".format(reco_trading_kwh, min_val, max_val))

# 이 둘의 평균값이 추천 거래 금액
reco_val = (min_val + max_val) / 2
print("{}원의 거래를 추천드립니다.".format(reco_val))

# 예상 수익은 추천 금액으로 거래 진행 시, 거래 금액 - 가구 2가 올라간 본 금액이 된다.
seller_profit = reco_val - min_val
buyer_profit = (buyer.basic + buyer.elec_rate)\
                                    - (after_buyer.basic + after_buyer.elec_rate)\
                                   - reco_val

print("거래 진행 시, 판매자의 예상 수익은 {}원 입니다.".format(seller_profit))
print("구매자의 수입은, 총 {}원 입니다.".format(buyer_profit))

# 서로 동등한 수익을 얻게되는 것,
# 공용부에는 어떤 변화가 찾아올까?

고객 가구 2은 판매 권장고객입니다.

현재 사용량은 150kWh이며, 이번달 예측 사용량은 170kWh입니다.

사용량 30kwh의 거래를 추천해드립니다.

거래 진행 시, 고객님의 사용량은 180kWh가 되며,
전기요금계는 14905.0원에서 17704.0원으로 2799.0원 증가합니다.


고객 가구 1와의 거래를 추천드립니다.
거래 진행 시, 구매자의 사용량은 390kWh가 되며,
전기요금계는 69152.0원에서 55961.0원으로 13191.0원 감소합니다.


거래 진행 시, 거래사용량 30kWh의 상품 가치는 2799.0원 ~ 13191.0원 입니다.
7995.0원의 거래를 추천드립니다.
5196.0
거래 진행 시, 판매자의 예상 수익은 5196.0원 입니다.
구매자의 수입은, 총 5196.0원 입니다.
